# INTERACTIVE SHELL

In [ ]:
import pandas as pd
from pandas import DataFrame
import sqlite3
import re
import json 
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
from cmd import Cmd


client = MongoClient('localhost', 27017)


with open('csvjson1.json', 'r') as myfile:
    data=myfile.read()

obj = json.loads(data)

mydb = client['classroom']

coll_class = mydb.classroom

coll_class = pd.DataFrame(list(coll_class.find()))

coll_class['TimeFrom'] = pd.to_datetime(coll_class['TimeFrom'], errors='coerce').dt.time
coll_class['TimeTo'] = pd.to_datetime(coll_class['TimeTo'], errors='coerce').dt.time

class MyPrompt(Cmd):
    
    prompt = 'CMS > '
    intro = "Welcome to Room Allocation System ! Type ? to list commands"
    


    
        
    
    def do_find(self, data):
        """Find the classroom where a certain course is taught
        """
        print("""
        For the first question: "What do you want to search?", 
        press C or c if you want to search for course name
        press I or i if you want to search for instructor name"
        """)
       
       
        day = input("""Please select a day. You must input the first letter of the day.
                       For example: M (Monday), T (Tuesday), W (Wednesday), T (Thursday), F(Friday) 
                    """)

        question = input("What do you want to search?: ")

        if question.lower() == "c":

            courseSearch = input("Which course do you want to search?: ")

            zz = coll_class.loc[(coll_class['CourseName'].str.contains(courseSearch, flags=re.I, regex=True)) & (coll_class['Days'].str.contains(day, flags=re.I, regex=True))]

            print(zz[['CourseName','Instructor','Days']])

        elif question.lower() == "i":

            instructorSearch = input("Which course do you want to search?: ")       

            xx = coll_class.loc[(coll_class['Instructor'].str.contains(instructorSearch, flags=re.I, regex=True)) & (coll_class['Days'].str.contains(day, flags=re.I, regex=True))]
            
            print(xx[['CourseName','Instructor','Days']])
            
        else:
        
            print("Input is not valid") 
        
    def do_provide(self, data):
        
        """Provide a list of available classrooms that are not 
       occupied given the day and time required.
        """
        print("""Select the time you want. Remember that time input must be H:M in 24hr format.
          """)
        ccc = True
        
        while ccc:
            
            try:
            
                From = pd.to_datetime(input('Time From: '))
                To = pd.to_datetime(input('Time To: '))
            
            except:
            
                print("Something else went wrong. Please check your time from input")
            
                continue
        
    
        
            zzz = coll_class[(((coll_class['TimeFrom']<= To.time()) & (coll_class['TimeFrom']>= From.time()))|((coll_class['TimeTo']<=To.time()) & (coll_class['TimeTo']>=From.time()))|((coll_class['TimeFrom']<=From.time()) & (coll_class['TimeTo']>=From.time()))|((coll_class['TimeFrom']<=To.time()) & (coll_class['TimeTo']>=To.time()))) & (coll_class['TimeFrom']!='')]




            day = input("""Please select a day. You must input the first letter of the day.
                           For example: M (Monday), T (Tuesday), W (Wednesday), R (Thursday), F(Friday) 
                        """)


            a = zzz[coll_class['Days'].str.contains(day, flags=re.I, regex=True)]

            roomOccupied = []
            for i in a['RoomName']:
                if i not in roomOccupied:
                    roomOccupied.append(i)
            df = pd.read_csv('roomAllocations.csv', na_values=["-"])
            wholeRoom = []
            for i in df['RoomName']:
                if i not in wholeRoom:
                    wholeRoom.append(i)


            main_list = np.setdiff1d(wholeRoom,roomOccupied)
            print("Here are the list of available room")
            if len(main_list) == 0:
                print('There is no room available')
            else:
                if len(main_list) == 1 and main_list[0] == 'nan':
                    print('There is no room available')
                    
                for i in main_list:
                    if i != 'nan':
                        print(i)
                
            ccc = False

        
        
    def do_exit(self, inp):
        '''Close the database and exit this program gracefully and '''
        print("Bye")
        client.close()
        return True
        

        
        
 
MyPrompt().cmdloop()
print("\nExit from CMS\n ")

Welcome to Room Allocation System ! Type ? to list commands
